In [1]:
import os
import json
import random
import statistics as stats

Source: https://www-cs-faculty.stanford.edu/~knuth/sgb.html

In [2]:
with open("sgb-words.txt", "r") as f:
    words = [w for w in f.read().split("\n") if len(w) == 5]
print(len(words))
print(words[:10])

5757
['which', 'there', 'their', 'about', 'would', 'these', 'other', 'words', 'could', 'write']


In [43]:
def test_wordle_guess(guess, answer):
    print(guess, answer)
    
    green = {}
    yellow = set()
    red = set()
    bad_duplicates = {}
    good_duplicates = {}
    
    current_chars = []
    
    for i in range(5):
        q_char = guess[i]
        current_chars.append(q_char)
        
        if (q_char in bad_duplicates) or (q_char in red):
            # Letter already used and not present, skip
            continue
        
        if not q_char in answer:
            # Character is not in word
            print(f"Character not present: {q_char}")
            red.add(q_char)
            continue
            
        if answer[i] == q_char:
            # Character is present, and in correct place
            print(f"Character correctly placed: {q_char}, {i}")
            green[i] = q_char
            continue
            
        # Check whether it is a duplicate letter
        guess_count = current_chars.count(q_char)
        answer_count = answer.count(q_char)
        print(f"Guess count: {guess_count}, Answer count: {answer_count}")
        if answer_count < guess_count:
            # Letter is a duplicate, and the subsequent one is not in answer
            print(f"Bad duplicate letter: {q_char}, {guess_count}")
            bad_duplicates[q_char] = guess_count
            continue
        elif guess_count > 1:
            good_duplicates[q_char] = guess_count
            
        # Letter must be present in answer, but in wrong place.
        print(f"Letter incorrectly placed: {q_char}, {i}")
        yellow.add( (q_char, i) )
        continue
        
    return (green, yellow, red, bad_duplicates, good_duplicates)


def wordle_guess(guess, answer):
    green = {}
    yellow = set()
    red = set()
    bad_duplicates = {}
    good_duplicates = {}
    
    current_chars = []
    
    for i in range(5):
        q_char = guess[i]
        current_chars.append(q_char)
        
        if (q_char in bad_duplicates) or (q_char in red):
            # Letter already used and not present, skip
            continue
        
        if not q_char in answer:
            # Character is not in word
            red.add(q_char)
            continue
            
        if answer[i] == q_char:
            # Character is present, and in correct place
            green[i] = q_char
            continue
            
        # Check whether it is a duplicate letter
        guess_count = current_chars.count(q_char)
        answer_count = answer.count(q_char)
        if answer_count < guess_count:
            # Letter is a duplicate, and the subsequent one is not in answer
            bad_duplicates[q_char] = guess_count
            continue
        elif guess_count > 1:
            good_duplicates[q_char] = guess_count
            
        # Letter must be present in answer, but in wrong place.
        yellow.add( (q_char, i) )
        continue
        
    return (green, yellow, red, bad_duplicates, good_duplicates)


def dummy_wordle_guess(guess, code):
    green = {}
    yellow = set()
    red = set()
    bad_duplicates = {}
    good_duplicates = {}
    
    current_chars = []
    
    for i in range(len(code)):
        char = guess[i]
        
        if code[i] in "Gg":
            green[i] = char
        
        elif code[i] in "Yy":
            yellow.add( (char, i) )
            if char in current_chars:
                good_duplicates[char] = current_chars.count(char) + 1
                
        elif code[i] in "Rr_":
            if char in current_chars:
                if char not in bad_duplicates:
                    bad_duplicates[char] = current_chars.count(char) + 1
            
            else :
                red.add(char)
            
        current_chars.append(char)
    
    return (green, yellow, red, bad_duplicates, good_duplicates)

In [44]:
info = test_wordle_guess("henna", "banal")
print(info)

henna banal
Character not present: h
Character not present: e
Character correctly placed: n, 2
Guess count: 2, Answer count: 1
Bad duplicate letter: n, 2
Guess count: 1, Answer count: 2
Letter incorrectly placed: a, 4
({2: 'n'}, {('a', 4)}, {'e', 'h'}, {'n': 2}, {})


In [46]:
print(wordle_guess("henna", "banal"))
print(dummy_wordle_guess("henna", "rrgry"))

({2: 'n'}, {('a', 4)}, {'e', 'h'}, {'n': 2}, {})
({2: 'n'}, {('a', 4)}, {'e', 'h'}, {'n': 2}, {})


In [5]:
def test_filter_words(word_list, green, yellow, red, bad_dupes, good_dupes):
    fwl = word_list
    
    print("Green:", green)
    print("Yellow:", yellow)
    print("Red:", red)
    
    for i in green:
        print("green", green[i])
        fwl = [w for w in fwl if w[i]==green[i]]
        
    print(fwl)
        
    for tup in yellow:
        char = tup[0]
        loc = tup[1]
        fwl = [w for w in fwl if (char in w) and (w[loc]!=char)]
        
    print(fwl)
    
    for r in red:
        print("red:", r)
        fwl = [w for w in fwl if r not in w]
        
    print(fwl)
        
    for b in bad_dupes:
        print("bad_dupes:", b)
        fwl = [w for w in fwl if w.count(b) < bad_dupes[b]]
        
    print(fwl)
        
    for g in good_dupes:
        print("good_dupes:", g)
        fwl = [w for w in fwl if w.count(g) >= good_dupes[g]]
        
    print(fwl)
    
    
def filter_words(word_list, green, yellow, red, bad_dupes, good_dupes):
    fwl = word_list
    
    for i in green:
        fwl = [w for w in fwl if w[i]==green[i]]
        
    for tup in yellow:
        char = tup[0]
        loc = tup[1]
        fwl = [w for w in fwl if (char in w) and (w[loc]!=char)]
    
    for r in red:
        fwl = [w for w in fwl if r not in w]
        
    for b in bad_dupes:
        fwl = [w for w in fwl if w.count(b) < bad_dupes[b]]
        
    for g in good_dupes:
        fwl = [w for w in fwl if w.count(g) >= good_dupes[g]]
        
    return fwl

In [6]:
if os.path.isfile("guess_values.txt"):
    with open("guess_values.txt", "r") as f:
        results = json.load(f)
else:
    results = []

total = len(words)
start_point = len(results)

try:
    for i in range(start_point, total):
        print(f"\r{i} / {total}", end="")
        guess_results = []
        guess = words[i]

        for j in range(total):
            answer = words[j]
            info = wordle_guess(guess, answer)
            guess_results.append(len(filter_words(words, *info)))

        score = sum(guess_results) / len(guess_results)
        results.append( (guess, score) )
        
except KeyboardInterrupt:
    pass
    
finally:
    with open("guess_values.txt", "w+") as f:
        json.dump(results, f)
    print("\nCurrent values saved.")


Current values saved.


In [7]:
len(results)

5757

In [8]:
sorted(results, key=lambda x: x[1], reverse=True)

[['ohhhh', 2492.348966475595],
 ['immix', 2445.5280528052804],
 ['gyppy', 2421.6913322911237],
 ['yukky', 2347.1269758554804],
 ['fuzzy', 2296.7576862949454],
 ['buzzy', 2182.3569567483064],
 ['yummy', 2173.8410630536737],
 ['mummy', 2161.462914712524],
 ['ahhhh', 2127.9423310752127],
 ['civic', 2097.3546986277574],
 ['puppy', 2083.715476810839],
 ['kudzu', 2016.0541948931734],
 ['pffft', 2003.2485669619593],
 ['vivid', 2003.1024839326037],
 ['fluff', 1998.348966475595],
 ['whooo', 1986.1966301893347],
 ['oxbow', 1979.133229112385],
 ['whizz', 1978.6539864512768],
 ['boffo', 1970.2433559145388],
 ['mamma', 1936.2072259857564],
 ['jiffy', 1924.8731978461003],
 ['duddy', 1901.039603960396],
 ['fizzy', 1897.0015633142261],
 ['buggy', 1820.8999478895257],
 ['infix', 1809.924092409241],
 ['quiff', 1806.859649122807],
 ['finif', 1800.5528921313185],
 ['jazzy', 1795.0795553239534],
 ['pygmy', 1793.4844537085287],
 ['chuff', 1792.806322737537],
 ['kooky', 1792.5348271669272],
 ['chuck', 1789.0

In [9]:
def random_word():
    value = int(random.random() * len(words))
    return words[value]


def get_word_scores(all_words, remaining_words, current_info=None):
    results = []
    for assumed_guess in all_words:
        guess_results = []
        for assumed_answer in remaining_words:
            guess_info = wordle_guess(assumed_guess, assumed_answer)
            assumed_info = merge_guess_info(guess_info, current_info)
            assumed_remaining_words = filter_words(remaining_words, (*assumed_info))
            guess_results.append(len(assumed_remaining_words))
        
        score = sum(guess_results) / len(guess_results)
        results.append( (assumed_guess, score) )
        
    return sorted(results, key=lambda x: x[1])


def merge_guess_info(info1, info2):
    default_info = ({}, set(), set(), {}, {})
    
    if info1 == None:
        info1 = default_info
    if info2 == None:
        info2 = default_info
        
    # Merge Greens
    green = {**info1[0], **info2[0]}
    
    # Merge Yellows
    yellow = info1[1].union(info2[1])
    
    # Merge Reds
    red = info1[2].union(info2[2])
    
    # Merge bad_dupes
    bad_dupes = {**info1[3], **info2[3]}
    
    # Merge good_dupes
    good_dupes = info1[4]
    for key in info2[4]:
        if key in good_dupes:
            good_dupes[key] = max(good_dupes[key], info2[4][key])
        else:
            good_dupes[key] = info2[4][key]

    return (green, yellow, red, bad_dupes, good_dupes)
    
        

def work_out_word(answer, print_guesses=False):
    guess = "tares"
    remaining_words = words
    guess_info = None
    guesses = []
    
    while True:
        if print_guesses:
            print(guess)
        guesses.append(guess)
        guess_info = merge_guess_info(wordle_guess(guess, answer), guess_info)
        remaining_words = filter_words(remaining_words, *guess_info)
        if len(remaining_words) == 1:
            guesses.append(remaining_words[0])
            if print_guesses:
                print(f"Final Guess: {remaining_words[0]}")
                print(f"Number of guesses: {len(guesses)}")
            return len(guesses)
        else:
            word_scores = get_word_scores(words, remaining_words, guess_info)
            guess = sorted(word_scores, key=lambda x: x[1])[0][0]  

#### Test 1

In [10]:
answer = random_word()
guess = "tares"
guess_info = merge_guess_info(wordle_guess(guess, answer), None)
remaining_words = filter_words(words, *guess_info)
remaining_words

['rules',
 'rises',
 'ropes',
 'cries',
 'rides',
 'roses',
 'roles',
 'robes',
 'dries',
 'urges',
 'frees',
 'pries',
 'fries',
 'roves',
 'ruses',
 'runes',
 'riles',
 'rubes',
 'bries',
 'rives',
 'rimes',
 'rices']

In [11]:
word_scores = get_word_scores(remaining_words, guess_info)
print(word_scores)
sorted(word_scores, key=lambda x: x[1])

[('rules', 5.0), ('rises', 5.0), ('ropes', 5.0), ('cries', 5.0), ('rides', 5.0), ('roses', 5.0), ('roles', 5.0), ('robes', 5.0), ('dries', 5.0), ('urges', 5.0), ('frees', 5.0), ('pries', 5.0), ('fries', 5.0), ('roves', 5.0), ('ruses', 5.0), ('runes', 5.0), ('riles', 5.0), ('rubes', 5.0), ('bries', 5.0), ('rives', 5.0), ('rimes', 5.0), ('rices', 5.0)]


[('rules', 5.0),
 ('rises', 5.0),
 ('ropes', 5.0),
 ('cries', 5.0),
 ('rides', 5.0),
 ('roses', 5.0),
 ('roles', 5.0),
 ('robes', 5.0),
 ('dries', 5.0),
 ('urges', 5.0),
 ('frees', 5.0),
 ('pries', 5.0),
 ('fries', 5.0),
 ('roves', 5.0),
 ('ruses', 5.0),
 ('runes', 5.0),
 ('riles', 5.0),
 ('rubes', 5.0),
 ('bries', 5.0),
 ('rives', 5.0),
 ('rimes', 5.0),
 ('rices', 5.0)]

In [12]:
guess2 = "nests"
guess_info = merge_guess_info(guess_info, wordle_guess(guess2, answer))
remaining_words = filter_words(remaining_words, *guess_info)
remaining_words

['rises', 'roses', 'ruses']

So if we only consider words that might be the answer (i.e. Hard Mode), we can end up in a situation where there is one letter missing that could be any of a handful of letters, with no way to eliminate them other than try them all. This can easily take us over the 6 guess limit. Therefore, we need to check the Expected Value of EVERY word in the full word list, for every guess. This will presumably take about 4 seconds per iteration, but will give us a much better chance.

EDIT: Okay, it's a lot more than 4 seconds per iteration. This may not be entirely feasible...

EDIT 2: Okay, it's fine. A run only takes up to about a minute.

#### Test 2

In [13]:
guess = "tares"
guess_info = merge_guess_info(wordle_guess(guess, answer), None)
remaining_words = filter_words(words, *guess_info)
remaining_words

['rules',
 'rises',
 'ropes',
 'cries',
 'rides',
 'roses',
 'roles',
 'robes',
 'dries',
 'urges',
 'frees',
 'pries',
 'fries',
 'roves',
 'ruses',
 'runes',
 'riles',
 'rubes',
 'bries',
 'rives',
 'rimes',
 'rices']

In [14]:
word_scores = get_word_scores(words, remaining_words, guess_info)
sorted(word_scores, key=lambda x: x[1])

[('build', 2.4545454545454546),
 ('broil', 2.909090909090909),
 ('limbo', 2.909090909090909),
 ('bison', 3.0),
 ('disco', 3.0),
 ('built', 3.090909090909091),
 ('bipod', 3.1818181818181817),
 ('boils', 3.272727272727273),
 ('guild', 3.272727272727273),
 ('pismo', 3.272727272727273),
 ('blind', 3.3636363636363638),
 ('blimp', 3.3636363636363638),
 ('proud', 3.5454545454545454),
 ('coils', 3.5454545454545454),
 ('foils', 3.5454545454545454),
 ('croup', 3.5454545454545454),
 ('doily', 3.5454545454545454),
 ('oiled', 3.5454545454545454),
 ('rosin', 3.5454545454545454),
 ('bruin', 3.5454545454545454),
 ('gismo', 3.5454545454545454),
 ('wilco', 3.5454545454545454),
 ('dildo', 3.5454545454545454),
 ('pilau', 3.5454545454545454),
 ('climb', 3.6363636363636362),
 ('pilot', 3.6363636363636362),
 ('olive', 3.6363636363636362),
 ('robin', 3.6363636363636362),
 ('viola', 3.6363636363636362),
 ('lucid', 3.6363636363636362),
 ('visor', 3.6363636363636362),
 ('voile', 3.6363636363636362),
 ('lisps', 3

In [15]:
guess2 = "lento"
guess_info = merge_guess_info(wordle_guess(guess2, answer), guess_info)
remaining_words = filter_words(remaining_words, *guess_info)
remaining_words

['ropes', 'roses', 'robes', 'roves']

In [16]:
word_scores = get_word_scores(words, remaining_words, guess_info)
sorted(word_scores, key=lambda x: x[1])

[('above', 1.5),
 ('basic', 1.5),
 ('verbs', 1.5),
 ('visit', 1.5),
 ('based', 1.5),
 ('prove', 1.5),
 ('basis', 1.5),
 ('brave', 1.5),
 ('vapor', 1.5),
 ('bases', 1.5),
 ('upset', 1.5),
 ('paste', 1.5),
 ('posts', 1.5),
 ('basin', 1.5),
 ('buses', 1.5),
 ('paved', 1.5),
 ('probe', 1.5),
 ('bosom', 1.5),
 ('bevel', 1.5),
 ('bushy', 1.5),
 ('bumps', 1.5),
 ('pivot', 1.5),
 ('pests', 1.5),
 ('posed', 1.5),
 ('plumb', 1.5),
 ('bison', 1.5),
 ('vases', 1.5),
 ('wasps', 1.5),
 ('beset', 1.5),
 ('gasps', 1.5),
 ('posse', 1.5),
 ('poses', 1.5),
 ('pussy', 1.5),
 ('pesos', 1.5),
 ('wisps', 1.5),
 ('bossy', 1.5),
 ('spasm', 1.5),
 ('bravo', 1.5),
 ('visor', 1.5),
 ('beeps', 1.5),
 ('baste', 1.5),
 ('viper', 1.5),
 ('vista', 1.5),
 ('busts', 1.5),
 ('baser', 1.5),
 ('vests', 1.5),
 ('rasps', 1.5),
 ('pasta', 1.5),
 ('visas', 1.5),
 ('wispy', 1.5),
 ('paves', 1.5),
 ('bused', 1.5),
 ('basks', 1.5),
 ('lisps', 1.5),
 ('bests', 1.5),
 ('blimp', 1.5),
 ('vises', 1.5),
 ('bumpy', 1.5),
 ('basil', 1.5

In [17]:
guess3 = "budge"
guess_info = merge_guess_info(wordle_guess(guess3, answer), guess_info)
remaining_words = filter_words(remaining_words, *guess_info)
remaining_words

['ropes', 'roses', 'roves']

In [18]:
word_scores = get_word_scores(words, remaining_words, guess_info)
sorted(word_scores, key=lambda x: x[1])

[('visit', 1.0),
 ('prove', 1.0),
 ('vapor', 1.0),
 ('upset', 1.0),
 ('paste', 1.0),
 ('posts', 1.0),
 ('paved', 1.0),
 ('pivot', 1.0),
 ('pests', 1.0),
 ('posed', 1.0),
 ('vases', 1.0),
 ('wasps', 1.0),
 ('gasps', 1.0),
 ('posse', 1.0),
 ('poses', 1.0),
 ('pussy', 1.0),
 ('pesos', 1.0),
 ('wisps', 1.0),
 ('spasm', 1.0),
 ('visor', 1.0),
 ('viper', 1.0),
 ('vista', 1.0),
 ('vests', 1.0),
 ('rasps', 1.0),
 ('pasta', 1.0),
 ('visas', 1.0),
 ('wispy', 1.0),
 ('paves', 1.0),
 ('lisps', 1.0),
 ('vises', 1.0),
 ('privy', 1.0),
 ('pesky', 1.0),
 ('cusps', 1.0),
 ('hasps', 1.0),
 ('pasty', 1.0),
 ('raspy', 1.0),
 ('poser', 1.0),
 ('apses', 1.0),
 ('peeve', 1.0),
 ('posit', 1.0),
 ('pesto', 1.0),
 ('vamps', 1.0),
 ('vapid', 1.0),
 ('veeps', 1.0),
 ('cuspy', 1.0),
 ('pushy', 1.0),
 ('poset', 1.0),
 ('pssst', 1.0),
 ('epsom', 1.0),
 ('spivs', 1.0),
 ('passe', 1.0),
 ('vapes', 1.0),
 ('poste', 1.0),
 ('vised', 1.0),
 ('pismo', 1.0),
 ('waspy', 1.0),
 ('pavan', 1.0),
 ('paver', 1.0),
 ('pasha', 1.0

#### Final Test

In [19]:
# This will possibly take up to 2 minutes to run
answer = random_word()
work_out_word(answer, True)
print(f"Answer: {answer}")

tares
sonly
Final Guess: soyas
Number of guesses: 3
Answer: soyas


#### Real World Test

In [20]:
# First Guess
guess_info = wordle_guess("tares","___ra")
print(guess_info)
remaining_words = filter_words(words, *guess_info)
print(len(remaining_words))
word_scores = get_word_scores(words, remaining_words, guess_info)
word_scores[:5]

({}, {('r', 2), ('a', 1)}, {'t', 'e', 's'}, {}, {})
102


[('blond', 6.0588235294117645),
 ('bland', 6.411764705882353),
 ('crony', 6.450980392156863),
 ('bronc', 6.450980392156863),
 ('irony', 6.490196078431373)]

In [21]:
# Second Guess
guess_info = merge_guess_info(guess_info, wordle_guess("blond", "___n_"))
print(guess_info)
remaining_words = filter_words(words, *guess_info)
print(remaining_words)
word_scores = get_word_scores(words, remaining_words, guess_info)
word_scores[:5]

({3: 'n'}, {('r', 2), ('a', 1)}, {'l', 'e', 'o', 'b', 's', 't', 'd'}, {}, {})
['crank', 'frank', 'franc', 'prank']


[('force', 1.0),
 ('facts', 1.0),
 ('chief', 1.0),
 ('crops', 1.0),
 ('faces', 1.0)]

In [22]:
guess_info = merge_guess_info(guess_info, wordle_guess("force", "rc   "))
print(guess_info)
remaining_words = filter_words(remaining_words, *guess_info)
remaining_words

({3: 'n'}, {('r', 2), ('c', 3), ('a', 1)}, {'e', 'b', 't', 'f', 'd', 'l', 'o', 's'}, {}, {})


['crank']

CRANK was correct.

---

In [47]:
guess_info = merge_guess_info(None, wordle_guess("tares", "re___"))
remaining_words = filter_words(words, *guess_info)
word_scores = get_word_scores(words, remaining_words, guess_info)
print(word_scores[:5])
print(len(remaining_words))

[('moire', 6.877551020408164), ('bride', 7.326530612244898), ('noire', 7.408163265306122), ('oldie', 7.714285714285714), ('guide', 7.877551020408164)]
98


In [48]:
guess_info = merge_guess_info(guess_info, wordle_guess("moire", "e__r_"))
remaining_words = filter_words(words, *guess_info)
word_scores = get_word_scores(words, remaining_words, guess_info)
print(word_scores[:5])
print(len(remaining_words))
print(remaining_words)
print(guess_info)

[('blend', 1.2), ('rebel', 1.2), ('beech', 1.2), ('bleed', 1.2), ('elude', 1.2)]
10
['every', 'clerk', 'query', 'henry', 'decry', 'heerd', 'leery', 'refry', 'exurb', 'beery']
({3: 'r'}, {('e', 4), ('e', 3), ('r', 2)}, {'a', 'o', 's', 't', 'i', 'm'}, {}, {})


In [27]:
guess_info = merge_guess_info(guess_info, wordle_guess("blend", "__e__"))
remaining_words = filter_words(words, *guess_info)
word_scores = get_word_scores(words, remaining_words, guess_info)
print(word_scores[:5])

[('there', 1.0), ('about', 1.0), ('would', 1.0), ('these', 1.0), ('could', 1.0)]


In [28]:
remaining_words

['every', 'query']

In [30]:
guess_info = merge_guess_info(guess_info, wordle_guess("there", "__er_"))
remaining_words = filter_words(words, *guess_info)

In [31]:
remaining_words

['query']

In [34]:
guess_info = wordle_guess("____u", "u    ")
remaining_words = filter_words(words, *guess_info)

In [35]:
remaining_words

['about',
 'would',
 'could',
 'sound',
 'found',
 'under',
 'house',
 'until',
 'study',
 'young',
 'music',
 'using',
 'group',
 'built',
 'quite',
 'round',
 'hours',
 'human',
 'build',
 'mouth',
 'value',
 'south',
 'equal',
 'guess',
 'cause',
 'quiet',
 'sugar',
 'rules',
 'units',
 'fruit',
 'nouns',
 'turns',
 'touch',
 'count',
 'quick',
 'truck',
 'upper',
 'usual',
 'proud',
 'court',
 'truth',
 'lunch',
 'doubt',
 'guide',
 'cloud',
 'funny',
 'hurry',
 'ought',
 'rough',
 'laugh',
 'occur',
 'outer',
 'ruler',
 'youth',
 'brush',
 'judge',
 'begun',
 'bound',
 'aloud',
 'pound',
 'fully',
 'flour',
 'curve',
 'queen',
 'union',
 'issue',
 'guard',
 'yours',
 'burst',
 'route',
 'lungs',
 'uncle',
 'mouse',
 'tough',
 'stuck',
 'truly',
 'swung',
 'lucky',
 'tubes',
 'wound',
 'trunk',
 'juice',
 'trust',
 'adult',
 'stuff',
 'thumb',
 'shout',
 'ruled',
 'crust',
 'fault',
 'ducks',
 'pulls',
 'drums',
 'drugs',
 'pause',
 'clues',
 'burnt',
 'sunny',
 'guest',
 'suits',


In [51]:
len(filter_words(words, *wordle_guess("___r_", "   r ")))

310

In [59]:
len(filter_words(words, *wordle_guess("oiast", "_____")))

402

In [62]:
g1 = wordle_guess("a____", "     ")
len(filter_words(words, *g1))

3576

In [63]:
g2 = wordle_guess("i____", "     ")
len(filter_words(words, *g2))

4218

In [64]:
len(filter_words(words, *merge_guess_info(g1, g2)))

2348

In [65]:
len(words)

5757

In [68]:
5757 - 2348

3409

#### Unified Class

In [170]:
class wordle_solver():
    def __init__(self, answer=None):
        self.answer = answer
        with open("sgb-words.txt", "r") as f:
            self.all_words = [w for w in f.read().split("\n") if len(w) == 5]
        
        
    def guess_from_answer(self, guess, answer=None):
        green = {}
        yellow = {}
        red = set()
        bad_duplicates = {}
        good_duplicates = {}

        current_chars = []

        for i in range(5):
            q_char = guess[i]
            current_chars.append(q_char)

            if (q_char in bad_duplicates) or (q_char in red):
                # Letter already used and not present, skip
                continue

            if not q_char in answer:
                # Character is not in word
                red.add(q_char)
                continue

            if answer[i] == q_char:
                # Character is present, and in correct place
                green[i] = q_char
                continue

            # Check whether it is a duplicate letter
            guess_count = current_chars.count(q_char)
            answer_count = answer.count(q_char)
            if answer_count < guess_count:
                # Letter is a duplicate, and the subsequent one is not in answer
                bad_duplicates[q_char] = guess_count
                continue
            elif guess_count > 1:
                good_duplicates[q_char] = guess_count

            # Letter must be present in answer, but in wrong place.
            if q_char not in yellow:
                yellow[q_char] = set()
            yellow[q_char].add(i)
            continue

        return wordle_guess_info(green, yellow, red, bad_duplicates, good_duplicates)
    
    
    def guess_from_code(self, guess, code):
        green = {}
        yellow = set()
        red = set()
        bad_duplicates = {}
        good_duplicates = {}

        current_chars = []

        for i in range(len(code)):
            char = guess[i]

            if code[i] in "Gg":
                green[i] = char

            elif code[i] in "Yy":
                if char not in yellow:
                    yellow[char] = set()
                yellow[char].add(i)
                if char in current_chars:
                    good_duplicates[char] = current_chars.count(char) + 1

            elif code[i] in "Rr_":
                if char in current_chars:
                    if char not in bad_duplicates:
                        bad_duplicates[char] = current_chars.count(char) + 1

                else :
                    red.add(char)

            current_chars.append(char)

        return wordle_guess_info(green, yellow, red, bad_duplicates, good_duplicates)
    
    
    def get_word_scores(self, remaining_words, current_info=None):
        results = []
        for assumed_guess in self.all_words:
            guess_results = []
            for assumed_answer in remaining_words:
                assumed_info = self.guess_from_answer(assumed_guess, assumed_answer) + current_info
                assumed_remaining_words = self.filter_words(remaining_words, assumed_info)
                guess_results.append(len(assumed_remaining_words))

            score = stats.mean(guess_results)
            results.append( (assumed_guess, score) )
        return sorted(results, key=lambda x: x[1])
    
    
    def filter_words(self, remaining_words, info):
        return [word for word in remaining_words if self.check_word(word, info)]
                    
        
    def check_word(self, word, info):
        for i in info.green:
            if word[i] != info.green[i]:
                return False
            
        for char in info.yellow:
            for i in info.yellow[char]:
                if word[i] == char:
                    return False
                
        for char in info.red:
            if char in word:
                return False
            
        for char in info.bad_duplicates:
            if word.count(char) >= info.bad_duplicates[char]:
                return False
            
        for char in info.good_duplicates:
            if word.count(char) < info.good_duplicates[char]:
                return False
            
        return True
    
    
    def solve(self, verbose=False):
        if answer == None:
            print("No answer specified. Cannot solve.")
            return
        
        guess = "tares"
        guesses = [guess]
        info = self.guess_from_answer(guess, self.answer)
        remaining_words = self.filter_words(self.all_words, info)
        
        while len(remaining_words) > 1:
            if verbose:
                print(f"Guess: {guess} - Remaining Words: {len(remaining_words)}")
            word_scores = self.get_word_scores(remaining_words, info)
            guess = word_scores[0][0]
            guesses.append(guess)
            info = self.guess_from_answer(guess, self.answer) + info
            remaining_words = self.filter_words(remaining_words, info)
            
        print(f"Answer: {remaining_words[0]}")
        print(f"Guesses: {len(guesses)}")
        return len(guesses)
    
    
    def manual_guess(self, guess):
        guess = guess.lower()
        response = ""
        for i in range(len(guess)):
            char = guess[i]
            if char == self.answer[i]:
                response += char.upper()
            elif char not in self.answer:
                response += "-"
            else:
                if answer.count(char) > response.lower().count(char):
                    response += char
                else:
                    response += "-"
        print(response)

In [171]:
ws = wordle_solver("banal")

In [172]:
ws.manual_guess("henna")

--N-a


In [174]:
ws.manual_guess("banan")

BANA-


In [151]:
class wordle_guess_info():
    def __init__(self, green={}, yellow={}, red=set(), bad_duplicates={}, good_duplicates={}):
        self.green = green
        self.yellow = yellow
        self.red = red
        self.bad_duplicates = bad_duplicates
        self.good_duplicates = good_duplicates
        
    def __add__(self, info):
        if info == None:
            return self
        elif type(info) != type(self):
            raise Exception(f"Can only add wordle_guess_info to similar objects, not to type {type(info)}.")
        
        green = {**self.green, **info.green}
        red = self.red.union(info.red)
        bad_duplicates = {**self.bad_duplicates, **info.bad_duplicates}
        
        # Good duplicate propogation        
        good_duplicates = self.good_duplicates
        for key in info.good_duplicates:
            if key in good_duplicates:
                good_duplicates[key] = max(self.good_duplicates[key],
                                           info.good_duplicates[key])
            else:
                good_duplicates[key] = info.good_duplicates[key]
        
        # Yellow propogation
        yellow = {}
        for d in (self.yellow, info.yellow):
            for char in d:
                if char not in yellow:
                    yellow[char] = set()
                yellow[char].update(d[char])
                
        return wordle_guess_info(green, yellow, red, bad_duplicates, good_duplicates)
                
    def __str__(self):
        return f"{self.green}, {self.yellow}, {self.red}, {self.bad_duplicates}, {self.good_duplicates}"

In [119]:
wgi+1

Exception: Can only add wordle_guess_info to similar objects, not to type <class 'int'>.

In [115]:
info = wordle_guess("henna", "banal")
info

({2: 'n'}, {('a', 4)}, {'e', 'h'}, {'n': 2}, {})

In [116]:
wgi = wordle_guess_info(info[0], {"a": set([4])}, info[2], info[3], info[4])
print(wgi)

{2: 'n'}, {'a': {4}}, {'e', 'h'}, {'n': 2}, {}


In [107]:
info2 = wordle_guess("manly", "banal")
info2

({1: 'a', 2: 'n'}, {('l', 3)}, {'m', 'y'}, {}, {})

In [111]:
wgi2 = wordle_guess_info(info2[0], {"l": set([3])}, info2[2], info2[3], info2[4])
print(wgi2)

{1: 'a', 2: 'n'}, {'l': {3}}, {'m', 'y'}, {}, {}


In [112]:
wgi3 = wgi + wgi2

In [113]:
print(wgi3)

{2: 'n', 1: 'a'}, {'a': {4}, 'l': {3}}, {'e', 'm', 'y', 'h'}, {'n': 2}, {}


In [130]:
print(info)

({2: 'n'}, {('a', 4)}, {'e', 'h'}, {'n': 2}, {})


In [131]:
print(avaad)

NameError: name 'avaad' is not defined

In [163]:
"a".upper()

'A'

In [164]:
"-----"

'-----'